In [ ]:
import ultraplot as uplt
import numpy as np
import h5py

## Check Colors

In [ ]:
def _run(zbin_ind):
    seed = 435
    nrand = 10_000

    ##################################################
    # het binning
    with h5py.File("/global/cfs/cdirs/des/y6kp-cats/final_version/desy6kp_cats_2024-11-07.hdf5") as f:
        zdnf_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/Z_MEAN"][:]
        zmin = zdnf_y6.min()
        zmax = zdnf_y6.max()

    zbin_name = f"{zbin_ind + 1}"
    print(f"zbin_name: {zbin_name}, zmin: {zmin}, zmax: {zmax}")

    ##################################################
    # read sim
    rng = np.random.default_rng(seed=seed)

    with h5py.File('/global/cfs/projectdirs/des/chto/Cardinal/v3/Cardinal-3_v2.0_Y6a_mastercat.h5') as f:
        maglim_inds = f['index/maglim/select'][:]
        zdnf_sim = f['catalog/bpz/unsheared/zmean_sof'][:][maglim_inds]
        mag_i_sim = f['catalog/gold/mag_i'][:][maglim_inds]
        msk = (
            np.isfinite(zdnf_sim)
            & (zdnf_sim >= zmin)
            & (zdnf_sim < zmax)
            & (17.5 < mag_i_sim)
            & (mag_i_sim < (4 * zdnf_sim + 18))
        )
        maglim_inds = maglim_inds[msk]

        keep = rng.choice(maglim_inds.shape[0], size=nrand, replace=False)
        maglim_inds = np.sort(maglim_inds[keep])

        zdnf_sim = f['catalog/bpz/unsheared/zmean_sof'][:][maglim_inds]
        z_sim = f['catalog/bpz/unsheared/z'][:][maglim_inds]
        mag_g_sim = f['catalog/gold/mag_g'][:][maglim_inds]
        mag_r_sim = f['catalog/gold/mag_r'][:][maglim_inds]
        mag_i_sim = f['catalog/gold/mag_i'][:][maglim_inds]
        mag_z_sim = f['catalog/gold/mag_z'][:][maglim_inds]

    msk = (
        np.isfinite(zdnf_sim)
        & (zdnf_sim >= zmin)
        & (zdnf_sim < zmax)
        & (17.5 < mag_i_sim)
        & (mag_i_sim < (4 * zdnf_sim + 18))
    )

    mag = mag_i_sim[msk]
    gmr = mag_g_sim[msk] - mag_r_sim[msk]
    rmi = mag_r_sim[msk] - mag_i_sim[msk]
    imz = mag_i_sim[msk] - mag_z_sim[msk]
    items_sim = [mag, gmr, rmi, imz]

    ##################################################
    # read data
    rng = np.random.default_rng(seed=seed)

    nrand = np.sum((
            np.isfinite(zdnf_sim)
            & (zdnf_sim >= zmin)
            & (zdnf_sim < zmax)
            & (17.5 < mag_i_sim)
            & (mag_i_sim < (4 * zdnf_sim + 18))
        )).astype(int)

    with h5py.File("/global/cfs/cdirs/des/y6kp-cats/final_version/desy6kp_cats_2024-11-07.hdf5") as f:
        keep = rng.choice(f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/BDF_MAG_I_CORRECTED"].shape[0], size=nrand, replace=False)
        mag_g_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/BDF_MAG_G_CORRECTED"][:][keep]
        mag_r_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/BDF_MAG_R_CORRECTED"][:][keep]
        mag_i_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/BDF_MAG_I_CORRECTED"][:][keep]
        mag_z_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/BDF_MAG_Z_CORRECTED"][:][keep]
        zdnf_y6 = f[f"desy6kp/maglim/tomo_bin_{zbin_ind}/Z_MEAN"][:][keep]

    msk = (
        np.isfinite(zdnf_y6)
        & (zdnf_y6 >= zmin)
        & (zdnf_y6 < zmax)
        & (17.5 < mag_i_y6)
        & (mag_i_y6 < (4 * zdnf_y6 + 18))
    )

    mag = mag_i_y6[msk]
    gmr = mag_g_y6[msk] - mag_r_y6[msk]
    rmi = mag_r_y6[msk] - mag_i_y6[msk]
    imz = mag_i_y6[msk] - mag_z_y6[msk]
    items_data = [mag, gmr, rmi, imz]

    print("number of objects (sim, data):", items_sim[0].shape[0], items_data[0].shape[0])

    ##################################################
    # make plots
    for items, data_name in zip(
        [items_sim, items_data],
        ["cardinal", "desy6"],
    ):
        fig, axs = uplt.subplots(nrows=4, ncols=4, share=0)
        axs.format(suptitle=f"bin {zbin_name} - " + data_name)

        ranges = []
        for item_sim, item_data in zip(items_sim, items_data):
            ranges.append(
                (
                    min(np.nanmin(item_sim), np.nanmin(item_data)),
                    max(np.nanmax(item_sim), np.nanmax(item_data))
                )
            )
        labels = ["i", "g-r", "r-i", "i-z"]
        reverse = [True, False, False, False]

        for i, itemi in enumerate(items):
            for j, itemj in enumerate(items):
                if i >= j:
                    if i == j:
                        axs[i, j].hist(itemi, bins=50, log=True, color="gray")
                        axs[i, j].format(xlim=ranges[i], ylim=(0.8, nrand // 2))

                        if reverse[i]:
                            axs[i, j].invert_xaxis()
                    else:
                        axs[i, j].hexbin(itemj, itemi, norm="log", gridsize=50, cmap="grays")
                        axs[i, j].format(xlim=ranges[j], ylim=ranges[i])

                        if reverse[j]:
                            axs[i, j].invert_xaxis()
                        if reverse[i]:
                            axs[i, j].invert_yaxis()

                    if i == len(items) - 1:
                        axs[i, j].set_xlabel(labels[j])
                    if j == 0 and i > 0:
                        axs[i, j].set_ylabel(labels[i])
                else:
                    axs[i, j].set_axis_off()

        fig.savefig(f"maglim_{zbin_name}_{data_name}.png", dpi=100)

In [ ]:
for zbin_ind in range(6):
    _run(zbin_ind)

## Check Performance of DNF

In [ ]:
import fitsio

seed = 435
zmin = 0.4
zmax = 0.55

In [ ]:
with fitsio.FITS("/global/cfs/cdirs/des/jvicente/y6/Y6_GOLD_2_2_DNF_match_maglim++_match_y6gold22_trainnov20.fits") as f:
    dmaglim = f[1].read()

zdnf_y6 = dmaglim["DNF_Z"]
z_y6 = dmaglim["Z"]
mag_i_y6 = dmaglim["BDF_MAG_I_CORRECTED"]

msk = (
    np.isfinite(zdnf_y6)
    & (zdnf_y6 >= zmin)
    & (zdnf_y6 < zmax)
    & (17.5 < mag_i_y6)
    & (mag_i_y6 < (4 * zdnf_y6 + 18))
)

dmaglim = dmaglim[msk]
zdnf_y6 = dmaglim["DNF_Z"]
z_y6 = dmaglim["Z"]
mag_i_y6 = dmaglim["BDF_MAG_I_CORRECTED"]

nrand = mag_i_y6.shape[0]

In [ ]:
rng = np.random.default_rng(seed=seed)

with h5py.File('/global/cfs/projectdirs/des/chto/Cardinal/v3/Cardinal-3_v2.0_Y6a_mastercat.h5') as f:
    maglim_inds = f['index/maglim/select'][:]
    zdnf_sim = f['catalog/bpz/unsheared/zmean_sof'][:][maglim_inds]
    mag_i_sim = f['catalog/gold/mag_i'][:][maglim_inds]
    msk = (
        np.isfinite(zdnf_sim)
        & (zdnf_sim >= zmin)
        & (zdnf_sim < zmax)
        & (17.5 < mag_i_sim)
        & (mag_i_sim < (4 * zdnf_sim + 18))
    )
    maglim_inds = maglim_inds[msk]

    keep = rng.choice(maglim_inds.shape[0], size=nrand, replace=False)
    maglim_inds = np.sort(maglim_inds[keep])

    zdnf_sim = f['catalog/bpz/unsheared/zmean_sof'][:][maglim_inds]
    z_sim = f['catalog/bpz/unsheared/z'][:][maglim_inds]
    mag_i_sim = f['catalog/gold/mag_i'][:][maglim_inds]

In [ ]:
def _make_plot(z, zdnf, name, bins=100, ax=None):
    lims = (zmin - 0.05, zmax + 0.05)
    binlocs = np.linspace(lims[0], lims[1], bins+1)
    binc = (binlocs[:-1] + binlocs[1:]) / 2.0
    lfracs = np.array([0.25, 0.5, 0.75, 0.9, 0.95])

    if ax is None:
        fig, axs = uplt.subplots(nrows=1, ncols=1, figsize=(5, 5))
        ax = axs[0, 0]

    ax.hexbin(
        z,
        zdnf,
        norm="log",
        gridsize=bins,
        cmap="grays",
        extent=lims + lims,
    )

    h, _, _ = np.histogram2d(
        z,
        zdnf,
        bins=binlocs,
    )
    h = h.T.astype(float)
    hlin = np.sort(h.flatten())
    hlinc = np.cumulative_sum(hlin) / np.sum(h)
    lvals = np.interp(1.0 - lfracs, hlinc, hlin)
    ax.contour(
        binc,
        binc,
        h,
        levels=lvals,
        cmap="rocket"
    )

    ax.format(
        ylim=lims,
        xlim=lims,
        ylabel="z DNF",
        xlabel="z",
    )
    px = ax.panel("t")

    px.hist(z, bins=binlocs, color="gray")
    px.format(xlim=lims)
    px = ax.panel("r")
    px.histh(zdnf, bins=binlocs, color="gray")
    px.format(ylim=lims)
    fig.format(suptitle=name)

    fname = (
        name
        .replace(" ", "_")
        .replace("[", "_")
        .replace("]", "_")
        .replace("=", "_")
        .replace("-", "_")
        .replace(",", "_")
        .replace("______", "_")
        .replace("_____", "_")
        .replace("____", "_")
        .replace("___", "_")
        .replace("__", "_")
    )
    fig.savefig(f"{fname}.png", dpi=100)

In [ ]:
_make_plot(z_sim, zdnf_sim, f"DNF z=[{zmin:0.2f}, {zmax:0.2f}] - cardinal", bins=50)
_make_plot(z_y6, zdnf_y6, f"DNF z=[{zmin:0.2f}, {zmax:0.2f}] - DES Y6", bins=50)